# Notes

## General

- Make a preformatted "Ship Information" text block to inject into prompts

## Missions

- Potentially assign stats to the ship such as "Notoriety" or "Reputation"
- Generate "Danger levels" or something of the sort for each mission
- Only assign missions with a danger level matching the ship's reputation.
- Potentially add more ship/crew information to the prompt?
- Currently Q/A is not working super well
- Figure out how to "do a mission"
    - Maybe first figure out what operating the ship will be like?

## The Ship

- What functionality does the ship have?
- What variables are tracked? (Heading, speed, shields, etc.)
- How are operations performed?
    - Captain commands subordinates who perform tasks

In [8]:
from langchain import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

In [9]:
model_bin = "/media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/llama/converted/llama_13b_ggml_q4_3.bin"
lora_bin = "/media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/SuperCOT-LoRA/13b/ggml/cutoff-2048/ggml-adapter-model.bin"

LLM = LlamaCpp(model_path=model_bin, lora_path=lora_bin, n_ctx=2048, n_threads=6, use_mmap=False, n_batch=512, temperature=1.0, top_p=0.6, stop=["\n\n### Input:"])

llama.cpp: loading model from /media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/llama/converted/llama_13b_ggml_q4_3.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 6 (mostly Q4_3)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size = 9534493.73 KB
llama_model_load_internal: mem required  = 11359.03 MB (+ 3216.00 MB per state)
....................................................................................................
llama_init_from_file: kv self size  = 3200.00 MB
llama_apply_lora_fro

In [15]:
user_vessel = "Carbide Cannon"
user_vessel_designation = "freighter"
user_vessel_capabilities = "trading, transporting small cargo loads, or transporting low-priority passengers"
current_location = "Starbase 453"
current_location_description = "a small starbase located near the edge of Federation space"

In [16]:
mission_statement_prompt = PromptTemplate.from_template("""### Instruction:
You are to act as a Star Trek storyboarding assistant. The user will request certain information, and you will generate succinct, creative, high-quality responses which meet the user's criteria.

### Input:
The {vessel} is a {vessel_designation} class ship designed for {vessel_capabilities}. It is currently docked at {location}, {location_description}. The crew of the {vessel} asks around {location} looking for a mission or task. Generate a brief summary of the mission/task they will receive.

### Response:
MISSION/TASK:
"""
)
chain = LLMChain(llm=LLM, prompt=mission_statement_prompt, verbose=True)

In [17]:
result = chain.run(vessel=user_vessel, vessel_designation=user_vessel_designation, vessel_capabilities=user_vessel_capabilities, location=current_location, location_description=current_location_description)
result



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
You are to act as a Star Trek storyboarding assistant. The user will request certain information, and you will generate succinct, creative, high-quality responses which meet the user's criteria.

### Input:
The Carbide Cannon is a freighter class ship designed for trading, transporting small cargo loads, or transporting low-priority passengers. It is currently docked at Starbase 453, a small starbase located near the edge of Federation space. The crew of the Carbide Cannon asks around Starbase 453 looking for a mission or task. Generate a brief summary of the mission/task they will receive.

### Response:
MISSION/TASK:


> Finished chain.



llama_print_timings:        load time =  4037.14 ms
llama_print_timings:      sample time =    30.21 ms /    81 runs   (    0.37 ms per run)
llama_print_timings: prompt eval time =  3717.17 ms /   166 tokens (   22.39 ms per token)
llama_print_timings:        eval time = 36784.92 ms /    80 runs   (  459.81 ms per run)
llama_print_timings:       total time = 40540.63 ms


'The crew of the Carbide Cannon is assigned to transport a high-priority diplomatic delegation from Starbase 453 to Starbase 71, located near the Romulan Neutral Zone. The delegation will discuss potential trade agreements and explore the possibility of establishing peaceful relations between the Federation and the Romulans.'

In [31]:
question_prompt = PromptTemplate.from_template("""### Instruction:
You are to act as a Star Trek storyboarding assistant. The user will request certain information, and you will generate succinct, creative, high-quality responses which meet the user's criteria.

### Input:
The {vessel} is a {vessel_designation} class ship designed for {vessel_capabilities}. It is currently docked at {location}, {location_description}. The crew of the {vessel} asked around {location} looking for a mission or task. They were asked to consider the following mission:

```
{mission_statement}
```

The crew has asked a question about the mission:

```
{question}
```

Generate a believable answer for the provided question. Use only the provided information about the {vessel} and its crew in your answer. Fabricate additional details about the mission or {location} if necessary. Be succinct.

### Response:
ANSWER: 
"""
)
question_chain = LLMChain(llm=LLM, prompt=question_prompt, verbose=True)

In [32]:
user_question = "Why should we take this mission? Our ship isn't equipped to transport high-priority individuals. Surely there must be a better crew for this job."
question_result = question_chain.run(vessel=user_vessel, vessel_designation=user_vessel_designation, vessel_capabilities=user_vessel_capabilities, location=current_location, location_description=current_location_description, mission_statement=result, question=user_question)
question_result



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
You are to act as a Star Trek storyboarding assistant. The user will request certain information, and you will generate succinct, creative, high-quality responses which meet the user's criteria.

### Input:
The Carbide Cannon is a freighter class ship designed for trading, transporting small cargo loads, or transporting low-priority passengers. It is currently docked at Starbase 453, a small starbase located near the edge of Federation space. The crew of the Carbide Cannon asked around Starbase 453 looking for a mission or task. They were asked to consider the following mission:

```
The crew of the Carbide Cannon is assigned to transport a high-priority diplomatic delegation from Starbase 453 to Starbase 71, located near the Romulan Neutral Zone. The delegation will discuss potential trade agreements and explore the possibility of establishing peaceful relations between the Federation and the Romulans.
```

T


llama_print_timings:        load time =  4037.14 ms
llama_print_timings:      sample time =    46.81 ms /   127 runs   (    0.37 ms per run)
llama_print_timings: prompt eval time =  5900.38 ms /   347 tokens (   17.00 ms per token)
llama_print_timings:        eval time = 58360.00 ms /   126 runs   (  463.17 ms per run)
llama_print_timings:       total time = 64321.65 ms


'The Carbide Cannon is a versatile freighter class ship, with ample cargo space for transporting small loads of goods and people. The crew is skilled in various areas, including engineering, navigation, diplomacy, and tactical operations. They are familiar with Starbase 453, which will help them navigate the complex diplomatic procedures at Starbase 71. Your mission is important to both the Federation and the Romulans, and it requires a crew with diverse skill sets and experience. The Carbide Cannon is uniquely suited for this assignment.'